In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import random
import lr_scheduler
import func

folder_path = "F:/code/barlow/UR5"
file_pairs = func.load_file_pairs(folder_path)

AUTO = tf.data.AUTOTUNE
CROP_TO = 32
SEED = 200

PROJECT_DIM = 2048
BATCH_SIZE = 32
EPOCHS = 50

In [2]:
random_500_similar, random_500_dissimilar, remaining_similar, remaining_dissimilar = func.split_and_shuffle_pairs(file_pairs, folder_path)

def process_data(data_list):
    return [
        [func.read_jpg_files(jpg_path), func.read_and_parse_npy_file(npy_path)]
        for jpg_path, npy_path in data_list
    ]

xtrain1 = process_data(random_500_similar)
xtrain2 = process_data(random_500_dissimilar)
xtest1 = process_data(remaining_similar)
xtest2 = process_data(remaining_dissimilar)

# label 1 - similar, 0 - disimilar
ytrain1 = [1] * 500
ytrain2 = [0] * 500
ytest1 = [1] *79
ytest2 = [0] *79

y_train = ytrain1 + ytrain2
y_test = ytest1 + ytest2
x_train = xtrain1 + xtrain2
x_test = xtest1 + xtest2

In [3]:
# Train data
t1 = np.array([item[0] for item in x_train])  # Lấy phần ảnh
t2 = np.array([item[1] for item in x_train])  # Lấy phần npy vector
train_ds = tf.data.Dataset.from_tensor_slices(((t1, t2),y_train))
train_ds = train_ds.map(
    lambda inputs, y: (func.combine_data(*inputs), y),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = (
    train_ds
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [4]:
# Test data
t3 = np.array([item[0] for item in x_test])  # Lấy phần ảnh
t4 = np.array([item[1] for item in x_test])  # Lấy phần npy vector
test_ds = tf.data.Dataset.from_tensor_slices(((t3, t4),y_test))
test_ds = test_ds.map(
    lambda inputs, y: (func.combine_data(*inputs), y),
    num_parallel_calls=tf.data.AUTOTUNE
)
test_ds = (
    test_ds
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [5]:
def create_mlp_encoder(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))

    # x1 = CONVO(inputs[...])
    # x2 = CONVO(inputs[-8])
    
    x = tf.keras.layers.Dense(512, activation="relu")(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dense(128, activation=None)(x)  # Projection head
    return tf.keras.Model(inputs, x)

# Tạo encoder mới
mlp_encoder = create_mlp_encoder(input_dim=16392)
# Sử dụng encoder MLP cho Barlow Twins
barlow_twins = func.BarlowTwins(encoder=mlp_encoder)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9)
barlow_twins.compile(optimizer=optimizer)
# barlow_twins.load_weights('bl.weights.h5')

In [6]:
backbone = tf.keras.Model(
    inputs=barlow_twins.encoder.input,
    outputs=barlow_twins.encoder.layers[-1].output  # output từ lớp Dense(128)
)
# Freeze 
backbone.trainable = False

# add classifier
inputs = tf.keras.layers.Input(shape=(16392,))
x = backbone(inputs, training=False)

#outputs = tf.keras.layers.Dense(10, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="relu")(x)

test_model = tf.keras.Model(inputs, outputs, name="test_model")


In [7]:
# Shuffle train_ds vs test_ds
seed = 226
train_ds = train_ds.shuffle(buffer_size=10, seed=seed, reshuffle_each_iteration=True)
test_ds = test_ds.shuffle(buffer_size=10, seed=seed, reshuffle_each_iteration=True)


In [8]:
# Compile model
test_model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00005, momentum=0.9)
)

history = test_model.fit(
    train_ds, validation_data=test_ds, epochs=100
)

_, test_acc = test_model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))


Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7774 - loss: 3.5880 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7493 - loss: 4.0413 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7364 - loss: 4.2490 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7569 - loss: 3.9188 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8062 - loss: 3.1233 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7594 - loss: 3.8775 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7633 - loss: 3.8159 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7726 - loss: 3.6654 - val_accuracy: 0.5000 - v

In [ ]:
# Predict on the test dataset
predictions = test_model.predict(test_ds)

# Extract the true labels from the test dataset
true_labels = np.concatenate([y for x, y in test_ds], axis=0)

# Print predictions and true labels
for i, prediction in enumerate(predictions):
    predicted_label = np.argmax(prediction)  # Get the predicted class index
    print(f"Sample {i}: Predicted Label = {predicted_label}, True Label = {true_labels[i]}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Sample 0: Predicted Label = 1, True Label = 1
Sample 1: Predicted Label = 1, True Label = 1
Sample 2: Predicted Label = 1, True Label = 1
Sample 3: Predicted Label = 1, True Label = 1
Sample 4: Predicted Label = 1, True Label = 1
Sample 5: Predicted Label = 1, True Label = 1
Sample 6: Predicted Label = 1, True Label = 1
Sample 7: Predicted Label = 1, True Label = 1
Sample 8: Predicted Label = 1, True Label = 1
Sample 9: Predicted Label = 1, True Label = 1
Sample 10: Predicted Label = 1, True Label = 1
Sample 11: Predicted Label = 1, True Label = 1
Sample 12: Predicted Label = 1, True Label = 1
Sample 13: Predicted Label = 1, True Label = 1
Sample 14: Predicted Label = 1, True Label = 1
Sample 15: Predicted Label = 1, True Label = 1
Sample 16: Predicted Label = 1, True Label = 1
Sample 17: Predicted Label = 1, True Label = 1
Sample 18: Predicted Label = 1, True Label = 1
Sample 19: Predicted Label = 1, True Label = 1
Sample 20: Predicted Label = 1, 